# Embedding API

## 1) Data Preprocessing

In [1]:
from langchain_community import embeddings
from langchain_ollama import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts  import ChatPromptTemplate

In [2]:
model_local = ChatOllama(model='phi4')

In [3]:
import json
from pprint import pprint

In [4]:
file1 = "./data/20250201/parsed_briefings_20250201.json"

with open(file1, "r", encoding="utf-8") as file:
    data = json.load(file)


In [11]:
documents = []
for briefing in data:
    page_content = {
        "type": briefing["type"],
        "title": briefing["title"],
        "content": briefing["content"],
        "date": briefing["date"]
        }
    
    metadata = {
        "url": briefing["url"]
        }
    
    newDocument = {
        "page_content": json.dumps(page_content, ensure_ascii=False),
        "metadata": metadata
        }
    documents.append(newDocument)
    

In [12]:
pprint(documents[0])

{'metadata': {'url': 'https://theminjoo.kr/main/sub/news//view.php?sno=0&brd=11&post=1209147&search='},
 'page_content': '{"type": "서면브리핑", "title": "[윤종군 원내대변인] AI 과학기술 발전의 조건이 윤석열 '
                 '탄핵·내란 특검법 중지라니 국민의힘은 제 정신입니까?", "content": "윤종군 원내대변인 '
                 '서면브리핑\\n■ AI 과학기술 발전의 조건이 윤석열 탄핵·내란 특검법 중지라니 국민의힘은 제 '
                 '정신입니까?\\n딥시크의 등장으로 전 세계가 충격에 휩싸였습니다. 딥시크는 전 세계적인 인공지능 경쟁에 '
                 '새로운 장을 열었습니다.\\n이에 더불어민주당 이재명 대표는 초당적인 협력과 함께 추경에 대대적인 인공지능 '
                 '개발 지원 예산을 담아주길 요청했습니다.\\n그러나 국민의힘은 추경안을 요청하기 전에 먼저 내란 특검법과 '
                 '내란 탄핵을 멈추라며 ‘내란 세력 수호’만 외치고 있습니다.\\n전 세계적인 AI 경쟁이 불타오르는데 이를 '
                 '추경에 담아 경제 성장의 동력을 찾을 생각은 하지 않고 정치공세에만 몰두하고 있습니다. 국민의힘은 국가의 '
                 '과학기술 발전과 미래 먹거리를 키워내는 일보다 정쟁과 정파가 먼저입니까?\\n‘반도체지원특별법’을 핑계 삼는 '
                 '것도 파렴치합니다. 국민의힘의 파행으로 국회 산자위가 열리지 못하고 제대로 논의도 못하고 있습니다. '
                 '국민의힘이 경제와 민생에 진심이라면 산자위 파행부터 해결하십시오.\\n우리가 내란과 정쟁으로 멈춰있는 동안 '
                 '세계는 달리고 있습니다. 정부는 

In [16]:
len(documents)

234

## 2) Data Insert

In [17]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [18]:
import time
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore

index_name = "test2"
pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=1024,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

In [19]:
index = pc.Index(index_name)
vector_store = PineconeVectorStore(index=index, embedding=embeddings.OllamaEmbeddings(model="bge-m3"))

/var/folders/yh/12scbgj57_d8xq3nw0klxfq00000gn/T/ipykernel_7838/3115346873.py:2: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  vector_store = PineconeVectorStore(index=index, embedding=embeddings.OllamaEmbeddings(model="bge-m3"))


In [20]:
from langchain_core.documents import Document

vectorDocuments = []
for document in documents:
    vectorDocument = Document(page_content=document["page_content"], metadata=document["metadata"])
    vectorDocuments.append(vectorDocument)

In [21]:
ids = [str(i) for i in range(264, len(vectorDocuments)+265)]
vector_store.add_documents(documents=vectorDocuments, ids=ids)

['264',
 '265',
 '266',
 '267',
 '268',
 '269',
 '270',
 '271',
 '272',
 '273',
 '274',
 '275',
 '276',
 '277',
 '278',
 '279',
 '280',
 '281',
 '282',
 '283',
 '284',
 '285',
 '286',
 '287',
 '288',
 '289',
 '290',
 '291',
 '292',
 '293',
 '294',
 '295',
 '296',
 '297',
 '298',
 '299',
 '300',
 '301',
 '302',
 '303',
 '304',
 '305',
 '306',
 '307',
 '308',
 '309',
 '310',
 '311',
 '312',
 '313',
 '314',
 '315',
 '316',
 '317',
 '318',
 '319',
 '320',
 '321',
 '322',
 '323',
 '324',
 '325',
 '326',
 '327',
 '328',
 '329',
 '330',
 '331',
 '332',
 '333',
 '334',
 '335',
 '336',
 '337',
 '338',
 '339',
 '340',
 '341',
 '342',
 '343',
 '344',
 '345',
 '346',
 '347',
 '348',
 '349',
 '350',
 '351',
 '352',
 '353',
 '354',
 '355',
 '356',
 '357',
 '358',
 '359',
 '360',
 '361',
 '362',
 '363',
 '364',
 '365',
 '366',
 '367',
 '368',
 '369',
 '370',
 '371',
 '372',
 '373',
 '374',
 '375',
 '376',
 '377',
 '378',
 '379',
 '380',
 '381',
 '382',
 '383',
 '384',
 '385',
 '386',
 '387',
 '388',


## 3) Chat Completion Test

In [22]:
results = vector_store.similarity_search(query="황정아",k=3)
print(len(results))
for doc in results:
    pprint(f"* {doc.page_content} [{doc.metadata}]")


3
('* {"소속": "중앙당", "이름": "황정아", "직책": "대변인", "슬로건": ""} [{\'국회\': '
 "'https://www.assembly.go.kr/members/22nd/HWANGJUNGA', '네이버블로그': "
 "'https://blog.naver.com/k0heaven', '유튜브': "
 "'https://www.youtube.com/@hwangjunga2024', '이미지': "
 "'https://theminjoo.kr/people/connect/people/1446/profile.jpg', '인스타그램': "
 "'https://www.instagram.com/junga1004/'}]")
('* {"소속": "국회의원", "이름": "황정아", "직책": "국회의원(대전, 유성구을)", "슬로건": ""} [{\'국회\': '
 "'https://www.assembly.go.kr/members/22nd/HWANGJUNGA', '네이버블로그': "
 "'https://blog.naver.com/k0heaven', '유튜브': "
 "'https://www.youtube.com/@hwangjunga2024', '이미지': "
 "'https://theminjoo.kr/people/connect/people/1446/profile.jpg', '인스타그램': "
 "'https://www.instagram.com/junga1004/'}]")
('* {"type": "서면브리핑", "title": "[황정아 대변인] 여론조사검증제도개선특위 발족, 건강한 공론장을 위한 과제들을 '
 '추진해 나가겠습니다", "content": "황정아 대변인 서면브리핑\\n■\\n여론조사검증제도개선특위 발족, 건강한 공론장을 위한 '
 '과제들을 추진해 나가겠습니다\\n\\n여론조사는 국민의 생각을 담아내는 중요한 공적 자원입니다. 정책과 정치의 방향을 결정하는 데 '
 '활용되며, 유권자가 올바른 판단을 내릴 수 있도록 돕는 민주사회의 필수

In [23]:
retriever = vector_store.as_retriever()

after_rag_template = """Refer to the content below and respond in Korean. Ensure that your response sounds natural, as if you are directly providing the answer yourself.:
{context}
Question: {question}
"""
after_rag_prompt = ChatPromptTemplate.from_template(after_rag_template)
after_rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | after_rag_prompt
    | model_local
    | StrOutputParser()
)

In [24]:
print(after_rag_chain.invoke("황정아가 최근에 얘기한거?"))


황정아 대변인은 최근 두 가지 중요한 이슈를 언급했습니다. 첫 번째는 여론조사검증제도의 개선입니다. 그녀는 최근 여론조사가 질문 문항, 응답률 및 편향성 측면에서 여러 논란이 제기되고 있다고 지적하며, 이를 해결하기 위해 여론조사검증제도를 개선할 것을 발표했습니다. 이는 국민의 신뢰를 회복시키고 정확한 공공 의견을 반영하는 데 중요하다고 강조했습니다.

두 번째로 언급된 것은 설 명절 기간 동안 발생한 내란 문제와 그에 따른 경제 및 민생 회복입니다. 황정아는 이러한 상황이 경제와 국민의 생활에 큰 부담을 주고 있다며, 이를 조기 종식하고 민생 경제를 살리겠다는 명확한 입장을 밝혔습니다. 그녀는 내란 세력들이 헌법과 법치에 대해 어긋나는 행동을 하고 있다며 비판하면서, 국민의 요구를 충족시키기 위한 책임 있는 조치가 필요하다고 강조했습니다.

이 두 가지 주제 모두 황정아 대변인은 정부와 시민사회에게 즉각적인 관심과 대응을 요구하며, 국가의 안정성과 국민의 복지를 최우선으로 여기고 있다는 입장을 분명히 했습니다.
